## 📌 Como Configurar um Chatbot Estruturado com RAG no LangChain

Agora, vamos configurar um pipeline de RAG para seu chatbot.

### Passo 1: Criando a Base de Conhecimento (LOCAL)

Agora, vamos configurar um pipeline de RAG para seu chatbot.



In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install tiktoken

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.schema import Document

# Criando documentos para o FAISS
documentos = [
    Document(page_content="Nosso suporte está disponível de segunda a sexta, das 9h às 18h."),
    Document(page_content="O reembolso pode ser solicitado em até 7 dias após a compra."),
    Document(page_content="Para falar com um atendente, envie um e-mail para suporte@empresa.com."),
]

# Usando embeddings locais com Ollama (modelo Mistral)
vectorstore = FAISS.from_documents(documentos, OllamaEmbeddings(model="mistral"))

print("Banco de vetores criado com sucesso usando Mistral!")


C:\Users\Eric\AppData\Local\Temp\ipykernel_26464\1179460957.py:13: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectorstore = FAISS.from_documents(documentos, OllamaEmbeddings(model="mistral"))


Banco de vetores criado com sucesso usando Mistral!


✅ Isso cria um banco de vetores com informações do suporte ao cliente.

### Passo 2: Criando o Chatbot RAG (LOCAL)

Agora, criamos um agente que busca respostas antes de responder:

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama  # Import correto para Ollama

llm = Ollama(model="mistral")

# Criando a cadeia de perguntas e respostas com recuperação de contexto
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

# Fazendo uma pergunta ao chatbot
pergunta = "Qual o horário de atendimento?"
resposta = qa.run(pergunta)

print("Resposta do Chatbot:", resposta)


C:\Users\Eric\AppData\Local\Temp\ipykernel_26464\405955736.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")
C:\Users\Eric\AppData\Local\Temp\ipykernel_26464\405955736.py:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = qa.run(pergunta)


Resposta do Chatbot:  O horário de atendimento é de segunda a sexta, das 9h às 18h.


### Passo 2.1: Criando o Chatbot RAG (VIA API)

Agora, criamos um agente que busca respostas antes de responder:

#### Conclusão

Qual LLM Escolher?

![taela_llm](taela_llm.png)

- ✅ Se precisar de um chatbot local, vá de Mistral 7B ou LLaMA 3.
- ✅ Se precisar de alta precisão, use GPT-4 Turbo ou Cohere API.

### Passo 2.1: Criando o Chatbot RAG (VIA API)

Agora, criamos um agente que busca respostas antes de responder:

In [3]:
import os
from langchain_community.llms import HuggingFaceHub
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings  # ou SentenceTransformersEmbeddings
from langchain.chains import RetrievalQA
from langchain.schema import Document


# 2. Instanciar o modelo LLM via Hugging Face
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={"temperature": 0.7, "max_length": 512},
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

# 3. Criar os documentos de conhecimento
documentos = [
    Document(page_content="Nosso suporte está disponível de segunda a sexta, das 9h às 18h."),
    Document(page_content="O reembolso pode ser solicitado em até 7 dias após a compra."),
    Document(page_content="Para falar com um atendente, envie um e-mail para suporte@empresa.com."),
]

# 4. Gerar embeddings e criar o FAISS
embedding_function = OllamaEmbeddings(model="mistral")  # ou outro modelo local
vectorstore = FAISS.from_documents(documentos, embedding=embedding_function)

# 5. Criar o pipeline RAG com RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# 6. Fazer uma pergunta
pergunta = "Como posso entrar em contato com o suporte?"
resposta = qa_chain.invoke(pergunta)

print("🔎 Resposta:", resposta["result"])
print("📄 Fonte(s):", [doc.page_content for doc in resposta["source_documents"]])


C:\Users\Eric\AppData\Local\Temp\ipykernel_26464\2655203033.py:10: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
C:\Users\Eric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Eric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarn

🔎 Resposta: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Nosso suporte está disponível de segunda a sexta, das 9h às 18h.

O reembolso pode ser solicitado em até 7 dias após a compra.

Para falar com um atendente, envie um e-mail para suporte@empresa.com.

Question: Como posso entrar em contato com o suporte?
Helpful Answer: Você pode entrar em contato com o suporte enviando um e-mail para suporte@empresa.com. O suporte está disponível de segunda a sexta, das 9h às 18h. Se você quiser solicitar um reembolso, faça isso dentro dos 7 primeiros dias após a compra.
📄 Fonte(s): ['Nosso suporte está disponível de segunda a sexta, das 9h às 18h.', 'O reembolso pode ser solicitado em até 7 dias após a compra.', 'Para falar com um atendente, envie um e-mail para suporte@empresa.com.']


❓ O embedding precisa ser feito com modelo local?
Não. Você pode usar modelos de embeddings remotos também! 🚀

LangChain permite que você escolha entre modelos locais ou via API para gerar os vetores de embedding. Tudo depende de custo, privacidade, velocidade e conectividade.

Se quiser usar embeddings da Hugging Face também (em vez do OllamaEmbeddings), troque para:



In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

✅ Resumo: posso usar...

![tabela.png](tabela.png)

#### 🚀 Agora você tem:

Um modelo remoto (HuggingFaceHub) ou local (Mistral).

Embeddings locais ou remotos.

Um pipeline completo de RAG.

Pronto para integrar com imagem, consulta com cliente final e muito mais!

## Passo 3: Personalização de Chatbots

Por que personalizar um chatbot?

- Humanização no atendimento.
- Manter a identidade e o tom da marca.
- Transmitir confiança e profissionalismo.

📌 Exemplo:

Compare as duas frases de uma I.A.:

❌ Genérico: “Não sei, tente outra pergunta.”

✅ Profissional: “Ainda não tenho essa informação, mas posso te encaminhar ao setor responsável. Deseja prosseguir?”

### 3.1 Criando um Perfil de Funcionário para o Chatbot 

Estratégia: Vamos criar um prompt-base com instruções claras para a LLM responder como um funcionário da empresa.

Exemplo de system_prompt personalizado:

In [ ]:
system_prompt = """
Você é um atendente virtual da empresa TechSolutions. 
Seu papel é responder de forma clara, educada e objetiva, 
como se fosse um funcionário treinado no setor de suporte ao cliente.

- Use uma linguagem formal e acolhedora.
- Se a resposta não for encontrada na base de conhecimento, oriente o usuário a entrar em contato pelo e-mail suporte@techsolutions.com.
- Nunca invente informações. Seja honesto e direto.
"""


✅ VERSÃO 1 Usando LLM local com Ollama (modelo mistral)

```python
from langchain_community.llms import Ollama
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate

# 1. Usando modelo local com Ollama (instale e rode com: `ollama run mistral`)
llm = Ollama(model="mistral")

# 2. Definindo o prompt personalizado
system_prompt = "Você é um assistente inteligente que sempre responde em português."

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])
...
```



✅ VERSÃO 2 – Usando LLM da Hugging Face (mistralai/Mistral-7B-Instruct-v0.2)

```python
from langchain_community.llms import HuggingFaceHub
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate
import os

# 1. Instanciar o LLM via Hugging Face Hub
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={"temperature": 0, "max_length": 512},
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

# 2. Prompt personalizado
system_prompt = "Você é um assistente inteligente que sempre responde em português."

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])
...
```

### 3.2 Personalizando as Respostas com a Base de Conhecimento



✅ VERSÃO 1 – Usando embeddings locais com Ollama (modelo mistral)

```python
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.schema import Document

# Documentos de entrada
documentos = [
    Document(page_content="Prezados clientes, nosso prazo de reembolso é de até 7 dias úteis após a solicitação."),
    Document(page_content="O atendimento funciona de segunda a sexta-feira, das 9h às 18h."),
    Document(page_content="Para segunda via de boleto, entre em contato pelo nosso portal ou e-mail.")
]

# Embeddings com modelo local via Ollama (precisa rodar: `ollama run mistral`)
embedding_function = OllamaEmbeddings(model="mistral")

# Criação do vetor FAISS
vectorstore = FAISS.from_documents(documentos, embedding=embedding_function)


✅ VERSÃO 2 – Usando embeddings da Hugging Face via API (ex: all-MiniLM-L6-v2)

```python
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.schema import Document
import os

# Configure sua chave da Hugging Face
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "sua_chave_aqui"

# Documentos de entrada
documentos = [
    Document(page_content="Prezados clientes, nosso prazo de reembolso é de até 7 dias úteis após a solicitação."),
    Document(page_content="O atendimento funciona de segunda a sexta-feira, das 9h às 18h."),
    Document(page_content="Para segunda via de boleto, entre em contato pelo nosso portal ou e-mail.")
]

# Embeddings via Hugging Face API (modelo leve e gratuito)
embedding_function = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Criação do vetor FAISS
vectorstore = FAISS.from_documents(documentos, embedding=embedding_function)

```


### 3.3 Finalização com montágem da cadeia com contexto + estilo profissional

✅ O chatbot agora responderá com base na base de conhecimento e no tom de voz definido.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chatbot = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template}
)


Código finalizado:

✅ VERSÃO 1 – Usando LLM local com Ollama + embeddings com Ollama

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

# 1. LLM local via Ollama
llm = Ollama(model="mistral", temperature=0)

# 2. Embeddings com modelo local via Ollama
embedding_function = OllamaEmbeddings(model="mistral")

# 3. Base de conhecimento
documentos = [
    Document(page_content="Prezados clientes, nosso prazo de reembolso é de até 7 dias úteis após a solicitação."),
    Document(page_content="O atendimento funciona de segunda a sexta-feira, das 9h às 18h."),
    Document(page_content="Para segunda via de boleto, entre em contato pelo nosso portal ou e-mail.")
]
vectorstore = FAISS.from_documents(documentos, embedding_function)

# 4. Memória da conversa
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 5. Prompt
system_prompt = "Você é um atendente educado e eficiente que responde sempre em português."
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

# 6. Cadeia de conversa com RAG
chatbot = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template}
)

# 7. Consulta
pergunta = "Como posso pedir a segunda via do boleto?"
resposta = chatbot.invoke({"question": pergunta})
print("🧠 Resposta:", resposta["answer"])


✅ VERSÃO 2 – Usando LLM da Hugging Face (via API) + embeddings via HF API

In [ ]:
import os
from langchain_community.llms import HuggingFaceHub
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

# 1. Configurar chave da Hugging Face
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "sua_chave_aqui"

# 2. LLM da Hugging Face (API)
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={"temperature": 0, "max_length": 512},
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

# 3. Embeddings via HF Inference API
embedding_function = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 4. Base de documentos
documentos = [
    Document(page_content="Prezados clientes, nosso prazo de reembolso é de até 7 dias úteis após a solicitação."),
    Document(page_content="O atendimento funciona de segunda a sexta-feira, das 9h às 18h."),
    Document(page_content="Para segunda via de boleto, entre em contato pelo nosso portal ou e-mail.")
]
vectorstore = FAISS.from_documents(documentos, embedding_function)

# 5. Memória de conversação
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 6. Prompt personalizado
system_prompt = "Você é um atendente educado e eficiente que responde sempre em português."
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

# 7. Cadeia conversacional RAG
chatbot = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt_template}
)

# 8. Pergunta do usuário
pergunta = "Como posso pedir a segunda via do boleto?"
resposta = chatbot.invoke({"question": pergunta})
print("🧠 Resposta:", resposta["answer"])
